In [1]:
#https://chatgpt.com/c/f4085671-33af-475d-b178-78182c65852f
#https://github.com/ogkalu2/comic-translate/blob/main/comic.py
#https://chatgpt.com/c/3a87cc64-ee76-4e0d-88fe-1421b54b81ba

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install necessary libraries
!pip install googletrans==3.1.0a0 ultralytics largestinteriorrectangle transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [4]:
!pip install jaconv

  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.3.4-py3-none-any.whl size=16416 sha256=31ee8ac36a21d5bd4c479e0faef2a65538a43df50217f6356654cd026f17c4a9
  Stored in directory: /root/.cache/pip/wheels/46/8f/2e/a730bf1fca05b33e532d5d91dabdf406c9b718ec85b01b1b54
Successfully built jaconv


In [25]:
!pip install fugashi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 10.3 MB/s eta 0:00:00


In [29]:
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=3cb27df3e87db0cab1cb1d58811cc9e729e791a09621c692b0f3ef5fd0622770
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [51]:
# Download NanumGothic font
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,563 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr

In [52]:
import re
import jaconv
from transformers import AutoFeatureExtractor, AutoTokenizer, VisionEncoderDecoderModel
import numpy as np
import torch
import cv2
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO
import largestinteriorrectangle as lir
import os
from googletrans import Translator
from google.colab import drive

In [58]:
def check_model_path(path):
    return os.path.isdir(path) and all(os.path.isfile(os.path.join(path, file)) for file in ['config.json', 'pytorch_model.bin', 'tokenizer_config.json'])

class MangaOcr:
    def __init__(self, model_path, device='cpu'):
        if os.path.isdir(model_path):
            # Load from local directory
            self.feature_extractor = AutoFeatureExtractor.from_pretrained(model_path, local_files_only=True)
            self.tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
            self.model = VisionEncoderDecoderModel.from_pretrained(model_path, local_files_only=True)
        else:
            # Load from HuggingFace Hub
            self.feature_extractor = AutoFeatureExtractor.from_pretrained(model_path)
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = VisionEncoderDecoderModel.from_pretrained(model_path)

        self.to(device)

    def to(self, device):
        self.model.to(device)

    @torch.no_grad()
    def __call__(self, img: np.ndarray):
        x = self.feature_extractor(img, return_tensors="pt").pixel_values.squeeze()
        x = self.model.generate(x[None].to(self.model.device))[0].cpu()
        x = self.tokenizer.decode(x, skip_special_tokens=True)
        x = post_process(x)
        return x

    def ocr_with_boxes(self, image, boxes):
        results = []
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cropped_img = image[y1:y2, x1:x2]
            text = self.__call__(cropped_img)
            results.append((text, box))
        return results

def post_process(text):
    text = ''.join(text.split())
    text = text.replace('…', '...')
    text = re.sub('[・.]{2,}', lambda x: (x.end() - x.start()) * '.', text)
    text = jaconv.h2z(text, ascii=True, digit=True)
    return text

def calculate_iou(rect1, rect2) -> float:
    x1 = max(rect1[0], rect2[0])
    y1 = max(rect1[1], rect2[1])
    x2 = min(rect1[2], rect2[2])
    y2 = min(rect1[3], rect2[3])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    rect1_area = (rect1[2] - rect1[0]) * (rect1[3] - rect1[1])
    rect2_area = (rect2[2] - rect2[0]) * (rect2[3] - rect2[1])

    union_area = rect1_area + rect2_area - intersection_area

    iou = intersection_area / union_area if union_area != 0 else 0

    return iou

def do_rectangles_overlap(rect1, rect2, iou_threshold: float = 0.2) -> bool:
    iou = calculate_iou(rect1, rect2)
    return iou >= iou_threshold

def does_rectangle_fit(bigger_rect, smaller_rect):
    x1, y1, x2, y2 = bigger_rect
    px1, py1, px2, py2 = smaller_rect

    left1, top1, right1, bottom1 = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
    left2, top2, right2, bottom2 = min(px1, px2), min(py1, py2), max(px1, py2), max(py1, py2)

    fits_horizontally = left1 <= left2 and right1 >= right2
    fits_vertically = top1 <= top2 and bottom1 >= bottom2

    return fits_horizontally and fits_vertically

def combine_results(bubble_detec_results: list, text_seg_results: list):
    bubble_bounding_boxes = np.array(bubble_detec_results[0].boxes.xyxy.cpu(), dtype="int")
    text_bounding_boxes = np.array(text_seg_results[0].boxes.xyxy.cpu(), dtype="int")

    segment_points = []
    if text_seg_results[0].masks is not None:
        segment_points = list(map(lambda a: a.astype("int"), text_seg_results[0].masks.xy))

    raw_results = []
    text_matched = [False] * len(text_bounding_boxes)

    if segment_points:
        for txt_idx, txt_box in enumerate(text_bounding_boxes):
            for bble_box in bubble_bounding_boxes:
                if does_rectangle_fit(bble_box, txt_box):
                    raw_results.append((txt_box, bble_box, segment_points[txt_idx], 'text_bubble'))
                    text_matched[txt_idx] = True
                    break
                elif do_rectangles_overlap(bble_box, txt_box):
                    raw_results.append((txt_box, bble_box, segment_points[txt_idx], 'text_free'))
                    text_matched[txt_idx] = True
                    break

    return raw_results

def adjust_contrast_brightness(img: np.ndarray, contrast: float = 1.0, brightness: int = 0):
    brightness += int(round(255 * (1 - contrast) / 2))
    return cv2.addWeighted(img, contrast, img, 0, brightness)

def ensure_gray(img: np.ndarray):
    if len(img.shape) > 2:
        return cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    return img.copy()

def make_bubble_mask(frame: np.ndarray):
    image = frame.copy()
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    stage_1 = cv2.drawContours(np.zeros_like(image), contours, -1, (255, 255, 255), thickness=2)
    stage_1 = cv2.bitwise_not(stage_1)
    stage_1 = cv2.cvtColor(stage_1, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(stage_1, 200, 255, cv2.THRESH_BINARY)
    num_labels, labels = cv2.connectedComponents(binary_image)
    largest_island_label = np.argmax(np.bincount(labels.flat)[1:]) + 1
    mask = np.zeros_like(image)
    mask[labels == largest_island_label] = 255
    _, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return adjust_contrast_brightness(mask, 100)

def bubble_interior_bounds(frame_mask: np.ndarray):
    gray = ensure_gray(frame_mask)
    ret, thresh = cv2.threshold(gray, 200, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    largest_contour = max(contours, key=cv2.contourArea)
    polygon = np.array([largest_contour[:, 0, :]])
    rect = lir.lir(polygon)
    x1, y1 = lir.pt1(rect)
    x2, y2 = lir.pt2(rect)
    return x1, y1, x2, y2

def fill_text(img, bbox, text, font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', color=(0, 0, 0)):
    from PIL import ImageFont, ImageDraw, Image
    x1, y1, x2, y2 = bbox

    # Create a blank image with white background
    blank_image = np.ones((y2 - y1, x2 - x1, 3), dtype=np.uint8) * 255

    # Convert the blank image to a PIL image
    img_pil = Image.fromarray(blank_image)
    draw = ImageDraw.Draw(img_pil)

    # Calculate the maximum font size that fits within the bounding box
    font_size = 1
    font = ImageFont.truetype(font_path, font_size)
    while True:
        text_size = draw.textsize(text, font=font)
        if text_size[0] <= (x2 - x1) and text_size[1] <= (y2 - y1):
            font_size += 1
            font = ImageFont.truetype(font_path, font_size)
        else:
            font_size -= 1
            font = ImageFont.truetype(font_path, font_size)
            text_size = draw.textsize(text, font=font)
            break

    # Draw the text on the blank image
    text_x = (x2 - x1 - text_size[0]) // 2
    text_y = (y2 - y1 - text_size[1]) // 2
    draw.text((text_x, text_y), text, font=font, fill=color)

    # Replace the text area in the original image with the blank image containing the text
    img[y1:y2, x1:x2] = np.array(img_pil)

def process_folder(folder_path, manga_ocr, bubble_model, text_model, translator, target_language='ko'):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            bubble_detec_results = bubble_model(img)
            text_seg_results = text_model(img)
            combined_results = combine_results(bubble_detec_results, text_seg_results)

            for text_box, bubble_box, segment_points, label in combined_results:
                x1, y1, x2, y2 = map(int, text_box)
                cropped_img = img[y1:y2, x1:x2]
                detected_text = manga_ocr(cropped_img)
                translated_text = translator.translate(detected_text, dest=target_language).text
                print(f"Detected Text: {detected_text}, Translated Text: {translated_text}, Bounding Box: {text_box}, Label: {label}")

                # Fill the bounding box with the translated text
                fill_text(img, text_box, translated_text)

            output_path = os.path.join(folder_path, f"translated_{filename}")
            cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [59]:
if __name__ == '__main__':
    # Setup models and translator
    manga_ocr_path = '/content/drive/MyDrive/imagetranslate/manga-ocr-base/'

    # Check if the path exists
    if not check_model_path(manga_ocr_path):
        # Try alternative paths
        alternative_paths = [
            '/content/drive/My Drive/imagetranslate/manga-ocr-base/',
            os.path.expanduser('~/imagetranslate/manga-ocr-base/'),
            './manga-ocr-base/'
        ]

        for path in alternative_paths:
            if check_model_path(path):
                manga_ocr_path = path
                break
        else:
            print("All local paths failed. Attempting to load from HuggingFace Hub.")
            manga_ocr_path = 'kha-white/manga-ocr-base'

    try:
        manga_ocr = MangaOcr(model_path=manga_ocr_path, device='cuda')
        print("MangaOcr initialized successfully!")
    except Exception as e:
        print(f"Error initializing MangaOcr: {str(e)}")
        print("Please ensure you have the correct model files or an internet connection to download from HuggingFace Hub.")

    bubble_model = YOLO("/content/drive/MyDrive/imagetranslate/weight/comic-speech-bubble-detector.pt")
    text_model = YOLO("/content/drive/MyDrive/imagetranslate/weight/comic-text-segmenter.pt")
    translator = Translator()

    # Path to the folder containing images
    folder_path = '/content/drive/MyDrive/imagetranslate/test/'

    # Process the folder
    process_folder(folder_path, manga_ocr, bubble_model, text_model, translator)

All local paths failed. Attempting to load from HuggingFace Hub.


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


MangaOcr initialized successfully!

0: 1024x672 18 text_bubbles, 62.7ms
Speed: 4.5ms preprocess, 62.7ms inference, 2.1ms postprocess per image at shape (1, 3, 1024, 672)

0: 1024x672 19 text_comics, 62.1ms
Speed: 4.0ms preprocess, 62.1ms inference, 2.5ms postprocess per image at shape (1, 3, 1024, 672)
Detected Text: ルフィお兄ちゃん好きなのよね！, Translated Text: 루피 오빠 좋아해!, Bounding Box: [218 473 282 542], Label: text_free
Detected Text: あー, Translated Text: 아, Bounding Box: [216 566 254 608], Label: text_free


<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tex

Detected Text: この子最近ルフィの顔を覚えて．．．, Translated Text: 이 아이 최근 루피의 얼굴을 기억해. . ., Bounding Box: [305 468 364 544], Label: text_free


...

<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tex

Detected Text: ＰＯＯＲＴＨＩＮＧ．, Translated Text: ＰＯＯＲＴＨＩＮＧ．, Bounding Box: [1776 1795 1929 1902], Label: text_bubble
Detected Text: ＯＨＰ, Translated Text: ＯＨＰ, Bounding Box: [ 800 1080  903 1138], Label: text_free


<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tex

Detected Text: ＰＦＦＴ．, Translated Text: ＰＦＦＴ．, Bounding Box: [ 311 1436  429 1502], Label: text_free


<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
<ipython-input-58-64944e5db923>:157: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tex